In [14]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
DATA_PATH = './data.json'

In [15]:
def load_data(data_path):
    with open(data_path, "r") as fp:
        data = json.load(fp)
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    return X, y

def prepare_datasets(test_size, validation_size):
    X, y = load_data(DATA_PATH)

    # split training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    # split training and validation sets
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # tensorflow expects as 3d array for CNNs (3d is like color/channel)
    X_train = X_train[..., np.newaxis] # 4d array -> (num samples, 130, 13, 1)
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test

def build_model(input_shape):
    # create model
    model = keras.Sequential()

    # 1st convolution layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 2nd convolution layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 3rd convolution layer
    model.add(keras.layers.Conv2D(32, (2, 2), activation="relu", input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten the output and feed it to the dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

    # output layer
    model.add(keras.layers.Dense(10, activation='softmax'))
    
    return model

def predict(model, X, y):
    X = X[np.newaxis, ...]
    # prediction is 2d array [[0.1, 0.2, ...]] for each genre for each sample
    prediction= model.predict(X)

    # extract index of max genre estimate
    predicted_index = np.argmax(prediction, axis=1)
    print("Expected index: {}, Predicted index: {}".format(y, predicted_index))

In [16]:
# create train, validation, and test sets
X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)
# build the CNN
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
model = build_model(input_shape)

# compile the network
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=['accuracy'])

# train
model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

# evaluation
test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy on test set is {}",format(test_accuracy)
      )

Epoch 1/30
188/188 [==============================] - 9s 41ms/step - loss: 2.4082 - accuracy: 0.2445 - val_loss: 1.8456 - val_accuracy: 0.3465
Epoch 2/30
188/188 [==============================] - 6s 33ms/step - loss: 1.8614 - accuracy: 0.3550 - val_loss: 1.5693 - val_accuracy: 0.4386
Epoch 3/30
188/188 [==============================] - 8s 41ms/step - loss: 1.6275 - accuracy: 0.4236 - val_loss: 1.4606 - val_accuracy: 0.4686
Epoch 4/30
188/188 [==============================] - 8s 42ms/step - loss: 1.4948 - accuracy: 0.4699 - val_loss: 1.3499 - val_accuracy: 0.5093
Epoch 5/30
188/188 [==============================] - 8s 45ms/step - loss: 1.3961 - accuracy: 0.4972 - val_loss: 1.3014 - val_accuracy: 0.5287
Epoch 6/30
188/188 [==============================] - 6s 33ms/step - loss: 1.3563 - accuracy: 0.5194 - val_loss: 1.2578 - val_accuracy: 0.5467
Epoch 7/30
188/188 [==============================] - 6s 33ms/step - loss: 1.2758 - accuracy: 0.5458 - val_loss: 1.2113 - val_accuracy: 0.5541

In [ ]:
# prediction
X = X_test[20]
y = y_test[20]
predict(model, X, y)

1/1 [==============================] - 0s 98ms/step
Expected index: 9, Predicted index: [7]
